## **A) Importações**

In [1]:
#pip install jupyter-dash

In [2]:
#pip install dash

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

from dash import dash, dcc, html
from dash.dependencies import Input, Output, State

from jupyter_dash import JupyterDash

In [2]:
df = pd.read_csv("variaveis_totais.csv")
ev = pd.read_excel("eventos_totais.xlsx")

In [9]:
df.head()

,Unnamed: 0,country,country_code,indicator,year,value
0,0,Albania,ALB,gdp_growth,1960,NaN
1,1,Albania,ALB,gdp_growth,1961,NaN
2,2,Albania,ALB,gdp_growth,1962,NaN
3,3,Albania,ALB,gdp_growth,1963,NaN
4,4,Albania,ALB,gdp_growth,1964,NaN


In [5]:
df_1 = df[df.indicator=='gdp_per_capita']

In [6]:
df_1['indicator1'] = 'gdp_per_capita_growth'
df_1['value1'] = np.nan

In [7]:
df_1.reset_index(inplace=True)

In [8]:
df_1.head(2)

,index,Unnamed: 0,country,country_code,indicator,year,value,indicator1,value1
0,2356,2356,Albania,ALB,gdp_per_capita,1960,NaN,gdp_per_capita_growth,NaN
1,2357,2357,Albania,ALB,gdp_per_capita,1961,NaN,gdp_per_capita_growth,NaN


In [10]:
for i in range(1,len(df_1)):
    if df_1.year[i] == 1960:
        pass
    else:
        df_1.value1[i] = (df_1.value[i]/df_1.value[i-1]-1)*100

In [17]:
df = df[['country','country_code','indicator','year','value']]
df_1 = df_1[['country','country_code','indicator1','year','value1']]

In [22]:
df_1 = df_1.rename(columns={'indicator1': 'indicator', 'value1': 'value'})

In [25]:
var = pd.concat([df[['country','country_code','indicator','year','value']],
                 df_1[['country','country_code','indicator','year','value']]
                ], axis=0)

In [27]:
var

,country,country_code,indicator,year,value
0,Albania,ALB,gdp_growth,1960,NaN
1,Albania,ALB,gdp_growth,1961,NaN
2,Albania,ALB,gdp_growth,1962,NaN
3,Albania,ALB,gdp_growth,1963,NaN
4,Albania,ALB,gdp_growth,1964,NaN
...,...,...,...,...,...
2351,United Kingdom,GBR,gdp_per_capita_growth,2017,-1.275334
2352,United Kingdom,GBR,gdp_per_capita_growth,2018,6.609779
2353,United Kingdom,GBR,gdp_per_capita_growth,2019,-1.291336
2354,United Kingdom,GBR,gdp_per_capita_growth,2020,-5.681137


In [28]:
# Encontre os anos mínimos e máximos no dataframe
min_year = var.year.min()
max_year = var.year.max()

In [29]:
ev=ev[['year_event','country_event','event_type','event']]

In [30]:
ev.head()

,year_event,country_event,event_type,event
0,1945,Malta,Diplomatic,Malta Conference (1945); President Franklin D....
1,1946,Albania,Political,The People's Assembly proclaimed the Socialist...
2,1946,Croatia,Constitutional,The 1946 Yugoslav Constitution came into force.
3,1946,France,Constitutional,France adopted the constitution of the Fourth ...
4,1946,Germany,Economic,The first of the Allied plans for German indus...


## **B) Testes de Visualização**

### **B.1) Visu c/ slider na escolha do tempo e não-delete do indicator**

In [19]:
del app1

NameError: name 'app1' is not defined

In [20]:
app1 = JupyterDash(__name__)

In [21]:
app1.layout = html.Div([
    html.Div([
        html.P("Select up to 3 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            #value=var.indicator.iloc[0],
            clearable=False,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),
    
    html.Div([
        dcc.RangeSlider(
            id='year-slider1',
            min=min_year,
            max=max_year,
            value=[2000, 2010],
            marks={str(year): {'label': str(year), 'style': {'transform': 'rotate(90deg)'}} for year in range(min_year, max_year+1)},
            step=None
        ),

        dcc.Graph(id="graph1"),

        dcc.Graph(id="graph2")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        dcc.RangeSlider(
            id='year-slider2',
            min=min_year,
            max=max_year,
            value=[2010, max_year],
            marks={str(year): {'label': str(year), 'style': {'transform': 'rotate(90deg)'}} for year in range(min_year, max_year+1)},
            step=None
        ),

        dcc.Graph(id="graph3"),

        dcc.Graph(id="graph4")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})

])

@app1.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 3:
        return value[:3]  # seleciona apenas os primeiros 3 países da lista
    return value

@app1.callback(Output("graph1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("year-slider1", "value")])
def display_graph1(selected_countries, selected_indicators, selected_years):
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= selected_years[0]) & (var['year'] <= selected_years[1])]
    grouped_var = filtered_var.groupby(['country', 'indicator']).mean().reset_index()
    fig1 = px.bar(grouped_var, x='country', y='value', color='country')
    return fig1

@app1.callback(Output("graph2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("year-slider1", "value")])
def display_graph2(selected_countries, selected_indicators, selected_years):
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= selected_years[0]) & (var['year'] <= selected_years[1])]
    fig2 = px.line(filtered_var, x='year', y='value', color='country')
    return fig2

@app1.callback(Output("graph3", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("year-slider2", "value")])
def display_graph3(selected_countries, selected_indicators, selected_years):
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= selected_years[0]) & (var['year'] <= selected_years[1])]
    grouped_var = filtered_var.groupby(['country', 'indicator']).mean().reset_index()
    fig3 = px.bar(grouped_var, x='country', y='value', color='country')
    return fig3

@app1.callback(Output("graph4", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("year-slider2", "value")])
def display_graph4(selected_countries, selected_indicators, selected_years):
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= selected_years[0]) & (var['year'] <= selected_years[1])]
    fig4 = px.line(filtered_var, x='year', y='value', color='country')
    return fig4

if __name__ == "__main__":
    app1.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


### **B.2) Visu c/ input box na escolha do tempo e não-delete do indicator**

In [35]:
del app2

In [36]:
app2 = JupyterDash(__name__)

In [37]:
app2.layout = html.Div([
    html.Div([
        html.P("Select up to 3 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            #value=var.indicator.iloc[0],
            clearable=False,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.P("Enter the start year:"),
        dcc.Input(
            id='start-year1',
            type='number',
            value=min_year
        ),
        html.P("Enter the end year:"),
        dcc.Input(
            id='end-year1',
            type='number',
            value=max_year
        ),
        dcc.Graph(id="graph1"),
        dcc.Graph(id="graph2")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.P("Enter the start year:"),
        dcc.Input(
            id='start-year2',
            type='number',
            value=min_year
        ),

        html.P("Enter the end year:"),
        dcc.Input(
            id='end-year2',
            type='number',
            value=max_year
        ),
        dcc.Graph(id="graph3"),
        dcc.Graph(id="graph4")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])

@app2.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 3:
        return value[:3]  # seleciona apenas os primeiros 3 países da lista
    return value

@app2.callback(Output("graph1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_graph1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'] == selected_indicators) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = filtered_var.groupby(['country', 'indicator']).mean().reset_index()
    fig1 = px.bar(grouped_var, x='country', y='value', color='country')
    return fig1

@app2.callback(Output("graph2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_graph2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'] == selected_indicators) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    fig2 = px.line(filtered_var, x='year', y='value', color='country')
    return fig2

@app2.callback(Output("graph3", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_graph3(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'] == selected_indicators) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = filtered_var.groupby(['country', 'indicator']).mean().reset_index()
    fig3 = px.bar(grouped_var, x='country', y='value', color='country')
    return fig3

@app2.callback(Output("graph4", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_graph4(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'] == selected_indicators) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    fig4 = px.line(filtered_var, x='year', y='value', color='country')
    return fig4

if __name__ == "__main__":
    app2.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


### **B.3) Visu c/ input box na escolha do tempo e podendo deletar o indicator**

In [44]:
del app3

In [45]:
app3 = JupyterDash(__name__)

In [46]:
app3.layout = html.Div([
    html.Div([
        html.P("Select up to 3 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year1',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year1',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="graph1"),
        dcc.Graph(id="graph2")
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year2',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year2',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="graph3"),
        dcc.Graph(id="graph4")
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])

@app3.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 3:
        return value[:3]  # seleciona apenas os primeiros 3 países da lista
    return value

@app3.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value

@app3.callback(Output("graph1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_graph1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    fig1 = px.bar(grouped_var, x='country', y='value', text='value', color='country')
    return fig1

@app3.callback(Output("graph2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_graph2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    fig2 = px.line(filtered_var, x='year', y='value', color='country')
    return fig2

@app3.callback(Output("graph3", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_graph3(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    fig3 = px.bar(grouped_var, x='country', y='value', text='value', color='country')
    return fig3

@app3.callback(Output("graph4", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_graph4(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    fig4 = px.line(filtered_var, x='year', y='value', color='country')
    return fig4

if __name__ == "__main__":
    app3.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


### **B.4) B.3 com mapa % diff**

In [182]:
del app4

In [183]:
app4 = JupyterDash(__name__)

In [184]:
app4.layout = html.Div([
    html.Div([
        html.P("Select up to 3 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year1',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year1',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="bar1"),
        dcc.Graph(id='map1', figure={}),
        dcc.Graph(id="line1")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year2',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year2',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="bar2"),
        dcc.Graph(id='map2', figure={}),
        dcc.Graph(id="line2")
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])

@app4.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 3:
        return value[:3]  # seleciona apenas os primeiros 3 países da lista
    return value

@app4.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value


# GRÁFICOS DO LADO ESQUERDO DA PÁGINA

@app4.callback(Output("bar1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_bar1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    bar1 = px.bar(grouped_var, x='country', y='value', text='value', color='country', title='Média da variável - Período 1')
    bar1.update_layout(title_x=0.5)
    return bar1

@app4.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map1 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 1",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0)
    map1.update_layout(height=500)
    map1.update_layout(title_x=0.5)
    return map1

@app4.callback(Output("line1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_line1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line1 = px.line(filtered_var, x='year', y='value', color='country', orientation='h', title='Linha do tempo da variável - Período 1')
    line1.update_layout(title_x=0.5)
    return line1


# GRÁFICOS DO LADO DIREITO DA PÁGINA

@app4.callback(Output("bar2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_bar2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    bar2 = px.bar(grouped_var, x='country', y='value', text='value', color='country', title='Média da variável - Período 2')
    bar2.update_layout(title_x=0.5)
    return bar2

@app4.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map2 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 2",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0)
    map2.update_layout(height=500)
    map2.update_layout(title_x=0.5)
    return map2

@app4.callback(Output("line2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line2 = px.line(filtered_var, x='year', y='value', color='country', title='Linha do tempo da variável - Período 2')
    line2.update_layout(title_x=0.5)
    return line2

if __name__ == "__main__":
    app4.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


### **B.5) B.4 com país NaN = cinza --> TENTATIVA**

In [349]:
del app5

In [350]:
app5 = JupyterDash(__name__)

In [351]:
app5.layout = html.Div([
    html.Div([
        html.P("Select up to 3 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year1',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year1',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="bar1"),
        dcc.Graph(id='map1', figure={}),
        dcc.Graph(id="line1")
    ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year2',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year2',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
            ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        dcc.Graph(id="bar2"),
        dcc.Graph(id='map2', figure={}),
        dcc.Graph(id="line2")
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])

@app5.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 3:
        return value[:3]  # seleciona apenas os primeiros 3 países da lista
    return value

@app5.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value


# GRÁFICOS DO LADO ESQUERDO DA PÁGINA

@app5.callback(Output("bar1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_bar1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    bar1 = px.bar(grouped_var, x='country', y='value', text='value', color='country', title='Média da variável - Período 1')
    bar1.update_layout(title_x=0.5)
    return bar1

@app5.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}
    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map1 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 1",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0)
    map1.update_layout(height=500)
    map1.update_layout(title_x=0.5)

    # Verifica se há valores NaN para cada país selecionado pelo usuário
    if any(var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]['value'].isnull()):
        # Cria uma lista com os códigos dos países sem dados
        no_data_country_codes = [filtered_var.groupby('country_code')['value'].mean().isna().idxmax()]
        # Adiciona uma camada com a cor cinza e a legenda "não há dados" para os países sem dados
        no_data_layer = go.Choropleth(locations=no_data_country_codes,
                                      z=[np.nan]*len(no_data_country_codes),
                                      hoverinfo='skip',
                                      showscale=False,
                                      colorscale=[[0, 'rgb(220,220,220)'], [1, 'rgb(220,220,220)']])
        map1.add_trace(no_data_layer)
    return map1

@app5.callback(Output("line1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_line1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line1 = px.line(filtered_var, x='year', y='value', color='country', orientation='h', title='Linha do tempo da variável - Período 1')
    line1.update_layout(title_x=0.5)
    return line1


# GRÁFICOS DO LADO DIREITO DA PÁGINA

@app5.callback(Output("bar2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_bar2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    bar2 = px.bar(grouped_var, x='country', y='value', text='value', color='country', title='Média da variável - Período 2')
    bar2.update_layout(title_x=0.5)
    return bar2

@app5.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Lida com valores nulos
    filtered_var.fillna(value=np.nan, inplace=True)
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map2 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 2",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0,
                         color_discrete_map={'np.nan': 'gray'})
    map2.update_layout(height=500)
    map2.update_layout(title_x=0.5)
    return map2

@app5.callback(Output("line2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line2 = px.line(filtered_var, x='year', y='value', color='country', title='Linha do tempo da variável - Período 2')
    line2.update_layout(title_x=0.5)
    return line2

if __name__ == "__main__":
    app5.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


In [43]:
del app6

In [44]:
app6 = JupyterDash(__name__, suppress_callback_exceptions=True)

In [45]:
#estilo = ['https://fonts.googleapis.com/css?family=Trebuchet+MS']
#app6 = JupyterDash(__name__, external_stylesheets = estilo)

In [46]:
### PÁGINA 1
# Criando o HTML geral
page_1_layout = html.Div([
    html.H2('Welcome to the country comparator!'),
    
    html.Div([
        html.P("Select up to 5 countries:"),
        dcc.Dropdown(
            id="dropdown-country",
            options=[{'label': str(val), 'value': val} for val in var.country.unique()],
            value=[],
            multi=True,
        ),
         html.Div(id="checkboxes-container-country")
    ]),

    html.Div([
        html.P("Select 1 indicator:"),
        dcc.Dropdown(
            id="dropdown-indicator",
            options=[{'label': val, 'value': val} for val in var.indicator.unique()],
            value=[],
            multi=True,
        ),
        html.Div(id="checkboxes-container-indicator")
    ]),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year1',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year1',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        
        dcc.Graph(id="line1"),
        dcc.Graph(id="bar1"),
        dcc.Graph(id='map1', figure={})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),

    html.Div([
        html.Div([
            html.P("Enter the start year:"),
            dcc.Input(
                id='start-year2',
                type='number',
                value=min_year,
                style={'width': '100px'}
                )
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
            html.P("Enter the end year:"),
            dcc.Input(
                id='end-year2',
                type='number',
                value=max_year,
                style={'width': '100px'}
                ),
        ], style={'width': '24%', 'display': 'inline-block', 'padding': '0 20'}),
        
        dcc.Graph(id="line2"),
        dcc.Graph(id="bar2"),
        dcc.Graph(id='map2', figure={})
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'})
])



#### FUNÇÕES DA PÁGINA 1
@app6.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")])
def validate_country_selection(value):
    if len(value) > 5:
        return value[:5]  # seleciona apenas os primeiros 3 países da lista
    return value

@app6.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")])
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  # seleciona apenas o primeiro 1 indicador da lista
    return value


# Gráficos do lado esquerdo
@app6.callback(Output("line1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_line1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line1 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 1')
    line1.update_layout(title_x=0.5)
    return line1


@app6.callback(Output("bar1", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")])
def display_bar1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    if selected_indicators:
        title = "Variação percentual em relação a " + selected_indicators[0] + " - Período 1"
    else:
        title = "Variação percentual - Período 1"
    bar1 = px.bar(grouped_var,
                  x='value',
                  y='country',
                  text='value',
                  color='country',
                  orientation='h',
                  title=title)
    bar1.update_layout(title_x=0.5)
    return bar1


@app6.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year1", "value"), Input("end-year1", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}
    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map1 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 1",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0)
    map1.update_layout(height=500)
    map1.update_layout(title_x=0.5)

    # Verifica se há valores NaN para cada país selecionado pelo usuário
    if any(var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]['value'].isnull()):
        # Cria uma lista com os códigos dos países sem dados
        no_data_country_codes = [filtered_var.groupby('country_code')['value'].mean().isna().idxmax()]
        # Adiciona uma camada com a cor cinza e a legenda "não há dados" para os países sem dados
        no_data_layer = go.Choropleth(locations=no_data_country_codes,
                                      z=[np.nan]*len(no_data_country_codes),
                                      hoverinfo='skip',
                                      showscale=False,
                                      colorscale=[[0, 'rgb(220,220,220)'], [1, 'rgb(220,220,220)']])
        map1.add_trace(no_data_layer)
    return map1


# Gráficos do lado direito
@app6.callback(Output("line2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_line2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    line2 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Linha do tempo - Período 2')
    line2.update_layout(title_x=0.5)
    return line2


@app6.callback(Output("bar2", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")])
def display_bar2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    grouped_var = round(filtered_var.groupby(['country', 'indicator']).mean(),2).reset_index()
    if selected_indicators:
        title = "Variação percentual em relação a " + selected_indicators[0] + " - Período 1"
    else:
        title = "Variação percentual - Período 1"
    bar2 = px.bar(grouped_var,
                  x='value',
                  y='country',
                  text='value',
                  color='country',
                  orientation='h',
                  title=title)
    bar2.update_layout(title_x=0.5)
    return bar2


@app6.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("start-year2", "value"), Input("end-year2", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, start_year, end_year):
    if not selected_countries:
        return {}    
    filtered_var = var[var['country'].isin(selected_countries) & (var['indicator'].isin(selected_indicators)) & (var['year'] >= start_year) & (var['year'] <= end_year)]
    # Cria um novo DataFrame apenas com as informações do primeiro país selecionado
    ref_country_var = filtered_var[filtered_var['country'] == selected_countries[0]]
    # Calcula a média dos valores para o primeiro país selecionado
    ref_country_mean = ref_country_var['value'].mean()
    # Cria uma nova coluna com o valor percentual em relação ao primeiro país
    filtered_var['value_pct'] = (filtered_var['value'] / ref_country_mean - 1) * 100
    # Lida com valores nulos
    filtered_var.fillna(value=np.nan, inplace=True)
    # Cria o mapa
    grouped_var = (round(filtered_var.groupby(['country', 'indicator', 'country_code'])['value_pct'].mean(), 2).reset_index())
    map2 = px.choropleth(grouped_var, 
                         locations="country_code",
                         color="value_pct",
                         hover_name="country",
                         projection="kavrayskiy7",
                         scope='europe',
                         title="Variação percentual em relação a " + selected_countries[0] + " - Período 2",
                         #range_color=[-50, 50],
                         color_continuous_scale='RdYlGn',
                         color_continuous_midpoint=0,
                         color_discrete_map={'np.nan': 'gray'})
    map2.update_layout(height=500)
    map2.update_layout(title_x=0.5)
    return map2



### PÁGINA 2
page_2_layout = html.Div([
    html.H2('What happened before/after some event?'),
    
    html.Div([
        html.P("Select 1 reference country:"),
        dcc.Dropdown(
            id="dropdown-ref_country",
            options=[{'label': str(val), 'value': val} for val in ev.country_event.unique()],
            value=[],
            multi=True,
        ),
        
        html.P("Select some event type(s):"),
        dcc.Dropdown(
            id='dropdown-event_type',
            options=[{'label': str(val), 'value': val} for val in ev.event_type.unique()],
            value=[],
            multi=True
        ),
        
        html.Table(id='table')
    ])
])



#### FUNÇÕES DA PÁGINA 2
@app6.callback(Output("dropdown-ref_country", "value"), [Input("dropdown-ref_country", "value")])
def validate_ref_country_selection(value):
    if len(value) > 1:
        return value[:1]
    return value


@app6.callback(Output("dropdown-event_type", "value"), [Input("dropdown-event_type", "value")])
def validate_event_type_selection(value):
    if len(value) > 1:
        return value[:1]
    return value





### LAYOUT GERAL
# Criando o HTML geral
tabs_layout = dcc.Tabs(id='tabs', value='tab-1', children=[
    dcc.Tab(label='COUNTRY COMPARATOR', value='tab-1', children=[page_1_layout]),
    dcc.Tab(label='IMPACT OF EVENTS', value='tab-2', children=[page_2_layout]),
])

app6.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        html.Img(src='infocountry_logo.png', style={'width':'362px', 'height':'77px', 'display': 'inline-block'}),
        tabs_layout
    ]),
    html.Div(id='page-content')
])

@app6.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page-2':
        return page_2_layout
    else:
        pass
    

### RODAR TUDO
if __name__ == "__main__":
    app6.run_server(port=8051, debug=True)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/


In [49]:
options=[{'label': str(val), 'value': val} for val in ev.country_event.unique()]

In [50]:
options

[{'label': 'Malta', 'value': 'Malta'},
 {'label': 'Albania', 'value': 'Albania'},
 {'label': 'Croatia', 'value': 'Croatia'},
 {'label': 'France', 'value': 'France'},
 {'label': 'Germany', 'value': 'Germany'},
 {'label': 'Italy', 'value': 'Italy'},
 {'label': 'Russia', 'value': 'Russia'},
 {'label': 'Bulgary', 'value': 'Bulgary'},
 {'label': 'Slovenia', 'value': 'Slovenia'},
 {'label': 'United Kingdom', 'value': 'United Kingdom'},
 {'label': 'Romania', 'value': 'Romania'},
 {'label': 'Greece', 'value': 'Greece'},
 {'label': 'Iceland', 'value': 'Iceland'},
 {'label': 'Belgium', 'value': 'Belgium'},
 {'label': 'Finland', 'value': 'Finland'},
 {'label': 'Luxembourg', 'value': 'Luxembourg'},
 {'label': 'Switzerland', 'value': 'Switzerland'},
 {'label': 'Cyprus', 'value': 'Cyprus'},
 {'label': 'Ireland', 'value': 'Ireland'},
 {'label': 'Latvia', 'value': 'Latvia'},
 {'label': 'Spain', 'value': 'Spain'},
 {'label': 'Portugal', 'value': 'Portugal'},
 {'label': 'Sweden', 'value': 'Sweden'},
 {'